In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time, pyautogui, re, json
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
driver = webdriver.Chrome()
driver.get("https://lactalis-smartit.onbmc.com/smartit/app/#/")
time.sleep(5)
pyautogui.hotkey("Esc")

In [4]:
driver.find_element(By.NAME, "AUTHENTICATION.LOGIN").send_keys("joaquim.junior.terceiro@br.lactalis.com")
driver.find_element(By.NAME, "AUTHENTICATION.PASSWORD").send_keys("L@ctali_S2024")
driver.find_element(By.NAME, "validateButton").click()

In [5]:
driver.find_element(By.XPATH, '//*[@id="main"]/div/div[1]/div[3]/div/button').click()
time.sleep(2)
total_chamados = driver.find_element(By.XPATH, '//*[@id="main"]/div/div[1]/div[3]/div[4]/h3/span').text
blocos_total = int(total_chamados)//75
def list_chamados(html):
    soup = BeautifulSoup(html, 'html.parser')
    chamados = soup.findAll('span', class_=re.compile('ng-binding ng-scope'))[7:]
    chamados_geral = []
    for chamado in chamados:
        chamados_geral.append(chamado.getText())
    return chamados_geral

def ResetBlocos(blocos_total):
    for bloco in range(0, blocos_total):
        driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[2]/div[2]/button').click()

In [6]:
chamados_tel = []
for bloco in range(0, blocos_total+1):
    if bloco == 0:
        chamados_tel.append(list_chamados(driver.page_source))
    elif driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[2]/div[2]/button').text == 'Próximo Bloco':
        driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[2]/div[2]/button').click()
        chamados_tel.append(list_chamados(driver.page_source))
    elif driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[2]/div[2]/button').text != 'Próximo Bloco':
        driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[2]/div[2]/button[2]').click()
        chamados_tel.append(list_chamados(driver.page_source))
ResetBlocos(blocos_total)

In [7]:
dados_chamados_geral = []

for n in range(0, len(chamados_tel)):
    for chamado in chamados_tel[n][7::10]:
        dados_dict = {}
        for item in chamado.split('\n- ')[1:-5]:
            try:
                chave, valor = item.split(':', 1)
                dados_dict[chave.strip()] = valor.strip()
            except:
                chave, valor = item.split('  ', 1)
                dados_dict[chave.strip()] = valor.strip()
                
        
        # Converter o dicionário para JSON
        dados_json = json.dumps(dados_dict, indent=4, ensure_ascii=False)
        dados_chamados_geral.append(dados_dict)
        
dados_geral = pd.DataFrame(dados_chamados_geral)
dados_geral

,Nome do solicitante,Solicitado para,Site (Filial),Diretoria,Empresa,Matrícula,Telefone de contato,Equipamento,Nome do computador,Problema,Descreva seu problema,question_templateName,Centro de Custo,question_Summary
0,GABRYEL SUKEJI SHIGUIHARA DE FARIA,GABRYEL SUKEJI SHIGUIHARA DE FARIA,BRA-028-ITAJAI,Finanças,Lactalis,041568814,: 65981026990,Desktop,BRLIAINB5X4VW2,Outros,: Preciso de um monitor suporte para usar como...,INC_HD2_Field Services - BR,4700101,NaN
1,GABRYEL SUKEJI SHIGUIHARA DE FARIA,GABRYEL SUKEJI SHIGUIHARA DE FARIA,BRA-028-ITAJAI,Finanças,Lactalis,041568814,: 65981026990,Notebook,BRLIAINB5X4VW2,Notebook não segura carga,NaN,INC_HD2_Field Services - BR,4700101,NaN
2,KAIO CESAR SILVA,KAIO CESAR SILVA,BRA-054-RECIFE-COMERCIAL,Supply,Lactalis,041564978,: 81998780499,Desktop,BRLRCEDV2H642,Computador Lento,NaN,INC_HD2_Field Services - BR,4208,NaN
3,Julio CESAR SANTIAGO DA COSTA,SARA HELENO FELIPE,BRA-078-PARA DE MINAS,Comercial,Itambé,91249,: 9636,Notebook,BRIPRSN863MPZ2,Computador Lento,NaN,INC_HD2_Field Services - BR,3900050,NaN
4,Gabriel Bastos Sampaio,Gabriel Bastos Sampaio,BRA-006-BARUERI,Supply,Lactalis,560314,: 11 9 4446-1391,Desktop,spo-069.br.lactadom.ad,Outros,: Memória cheia. Impossibilitando o uso. Usuár...,INC_HD2_Field Services - BR,7204100,BR_71 - Problema no equipamento
5,Marcieli Zampieron,Marcieli Zampieron,BRA-049-SÃO MIGUEL DO OESTE,Captação,Lactalis,558777,: 4991546527,Desktop,BRLSGED1323QK2,Outros,": TRoquei de cpu do computador, e esta aparece...",INC_HD2_Field Services - BR,4506000,BR_71 - Problema no equipamento
6,Marcieli Zampieron,Marcieli Zampieron,BRA-049-SÃO MIGUEL DO OESTE,Captação,Lactalis,558777,: 4991546527,Desktop,BRLSGED1323QK2,Tela azul,NaN,INC_HD2_Field Services - BR,4506000,NaN
7,Jose Eduardo Souto Alen,Jose Eduardo Souto Alen,BRA-027-IJUÍ,Captação,Lactalis,559216,: 5533054405,Notebook,IJINJ7PZWQ,Computador Lento,NaN,INC_HD2_Field Services - BR,4004800,NaN
8,Gustavo Humberto Kluser,Gustavo Humberto Kluser,BRA-028-ITAJAI,Finanças,Lactalis,442572,: 47996658453,Notebook,BRJ342B1N7,Tela do notebook danificada,NaN,INC_HD2_Field Services - BR,7700105,NaN
9,FABIO AUGUSTO SILVA MORAIS LARA,FABIO AUGUSTO SILVA MORAIS LARA,BRA-078-PARA DE MINAS,Industrial,Itambé,86041,: 37 991104065,Notebook,S/N BRJ401M9,Outros,: Ao colocar o computador na mochila o ziper d...,INC_HD2_Field Services - BR,3013400,NaN


In [8]:
def dados_chamados(id:int):
    dados_chamado = []
    for tel in chamados_tel:
        dados_chamado.extend(tel[id::10])
    return dados_chamado

df = pd.DataFrame({
    'Número do Chamado': dados_chamados(0),
    'Cliente': dados_chamados(2),
    'Filial': dados_chamados(3),
    'Grupo designado': dados_chamados(8),
    'Tipo': dados_chamados(5),
    'Status': dados_chamados(6),
    'Data de criação': dados_chamados(1),
})

df

,Número do Chamado,Cliente,Filial,Grupo designado,Tipo,Status,Data de criação
0,INC000001048072,GABRYEL SUKEJI SHIGUIHARA DE FARIA,BRA-028-ITAJAI,Field Services - BR,BR_71 - Problema no equipamento,Pendente,29 de jul de 2024 09:08:39
1,INC000001048054,GABRYEL SUKEJI SHIGUIHARA DE FARIA,BRA-028-ITAJAI,Field Services - BR,BR_71 - Problema no equipamento,Designado,29 de jul de 2024 07:57:27
2,INC000001047577,KAIO CESAR SILVA,BRA-054-RECIFE-COMERCIAL,Field Services - BR,BR_71 - Problema no equipamento,Pendente,26 de jul de 2024 16:09:21
3,INC000001047409,Sara HELENO FELIPE,BRA-078-PARA DE MINAS,Field Services - BR,BR_71 - Problema no equipamento,Pendente,26 de jul de 2024 10:29:51
4,INC000001046959,Gabriel Bastos Sampaio,BRA-006-BARUERI,Field Services - BR,BR_71 - Problema no equipamento,Pendente,25 de jul de 2024 17:23:03
5,INC000001045622,Marcieli Zampieron,BRA-049-SÃO MIGUEL DO OESTE,Field Services - BR,BR_71 - Problema no equipamento,Pendente,23 de jul de 2024 11:53:33
6,INC000001045372,Marcieli Zampieron,BRA-049-SÃO MIGUEL DO OESTE,Field Services - BR,BR_71 - Problema no equipamento,Pendente,23 de jul de 2024 09:34:38
7,INC000001045363,Jose Eduardo Souto Alen,BRA-027-IJUÍ,Field Services - BR,BR_71 - Problema no equipamento,Pendente,23 de jul de 2024 09:24:55
8,INC000001038935,Gustavo Humberto Kluser,BRA-028-ITAJAI,Field Services - BR,BR_71 - Problema no equipamento,Pendente,15 de jul de 2024 09:01:47
9,INC000001030864,FABIO AUGUSTO SILVA MORAIS LARA,BRA-078-PARA DE MINAS,Field Services - BR,BR_71 - Problema no equipamento,Pendente,26 de jun de 2024 13:16:14


In [9]:
df_pronto = pd.concat([df, dados_geral], axis=1)
df_pronto

,Número do Chamado,Cliente,Filial,Grupo designado,Tipo,Status,Data de criação,Nome do solicitante,Solicitado para,Site (Filial),...,Empresa,Matrícula,Telefone de contato,Equipamento,Nome do computador,Problema,Descreva seu problema,question_templateName,Centro de Custo,question_Summary
0,INC000001048072,GABRYEL SUKEJI SHIGUIHARA DE FARIA,BRA-028-ITAJAI,Field Services - BR,BR_71 - Problema no equipamento,Pendente,29 de jul de 2024 09:08:39,GABRYEL SUKEJI SHIGUIHARA DE FARIA,GABRYEL SUKEJI SHIGUIHARA DE FARIA,BRA-028-ITAJAI,...,Lactalis,041568814,: 65981026990,Desktop,BRLIAINB5X4VW2,Outros,: Preciso de um monitor suporte para usar como...,INC_HD2_Field Services - BR,4700101,NaN
1,INC000001048054,GABRYEL SUKEJI SHIGUIHARA DE FARIA,BRA-028-ITAJAI,Field Services - BR,BR_71 - Problema no equipamento,Designado,29 de jul de 2024 07:57:27,GABRYEL SUKEJI SHIGUIHARA DE FARIA,GABRYEL SUKEJI SHIGUIHARA DE FARIA,BRA-028-ITAJAI,...,Lactalis,041568814,: 65981026990,Notebook,BRLIAINB5X4VW2,Notebook não segura carga,NaN,INC_HD2_Field Services - BR,4700101,NaN
2,INC000001047577,KAIO CESAR SILVA,BRA-054-RECIFE-COMERCIAL,Field Services - BR,BR_71 - Problema no equipamento,Pendente,26 de jul de 2024 16:09:21,KAIO CESAR SILVA,KAIO CESAR SILVA,BRA-054-RECIFE-COMERCIAL,...,Lactalis,041564978,: 81998780499,Desktop,BRLRCEDV2H642,Computador Lento,NaN,INC_HD2_Field Services - BR,4208,NaN
3,INC000001047409,Sara HELENO FELIPE,BRA-078-PARA DE MINAS,Field Services - BR,BR_71 - Problema no equipamento,Pendente,26 de jul de 2024 10:29:51,Julio CESAR SANTIAGO DA COSTA,SARA HELENO FELIPE,BRA-078-PARA DE MINAS,...,Itambé,91249,: 9636,Notebook,BRIPRSN863MPZ2,Computador Lento,NaN,INC_HD2_Field Services - BR,3900050,NaN
4,INC000001046959,Gabriel Bastos Sampaio,BRA-006-BARUERI,Field Services - BR,BR_71 - Problema no equipamento,Pendente,25 de jul de 2024 17:23:03,Gabriel Bastos Sampaio,Gabriel Bastos Sampaio,BRA-006-BARUERI,...,Lactalis,560314,: 11 9 4446-1391,Desktop,spo-069.br.lactadom.ad,Outros,: Memória cheia. Impossibilitando o uso. Usuár...,INC_HD2_Field Services - BR,7204100,BR_71 - Problema no equipamento
5,INC000001045622,Marcieli Zampieron,BRA-049-SÃO MIGUEL DO OESTE,Field Services - BR,BR_71 - Problema no equipamento,Pendente,23 de jul de 2024 11:53:33,Marcieli Zampieron,Marcieli Zampieron,BRA-049-SÃO MIGUEL DO OESTE,...,Lactalis,558777,: 4991546527,Desktop,BRLSGED1323QK2,Outros,": TRoquei de cpu do computador, e esta aparece...",INC_HD2_Field Services - BR,4506000,BR_71 - Problema no equipamento
6,INC000001045372,Marcieli Zampieron,BRA-049-SÃO MIGUEL DO OESTE,Field Services - BR,BR_71 - Problema no equipamento,Pendente,23 de jul de 2024 09:34:38,Marcieli Zampieron,Marcieli Zampieron,BRA-049-SÃO MIGUEL DO OESTE,...,Lactalis,558777,: 4991546527,Desktop,BRLSGED1323QK2,Tela azul,NaN,INC_HD2_Field Services - BR,4506000,NaN
7,INC000001045363,Jose Eduardo Souto Alen,BRA-027-IJUÍ,Field Services - BR,BR_71 - Problema no equipamento,Pendente,23 de jul de 2024 09:24:55,Jose Eduardo Souto Alen,Jose Eduardo Souto Alen,BRA-027-IJUÍ,...,Lactalis,559216,: 5533054405,Notebook,IJINJ7PZWQ,Computador Lento,NaN,INC_HD2_Field Services - BR,4004800,NaN
8,INC000001038935,Gustavo Humberto Kluser,BRA-028-ITAJAI,Field Services - BR,BR_71 - Problema no equipamento,Pendente,15 de jul de 2024 09:01:47,Gustavo Humberto Kluser,Gustavo Humberto Kluser,BRA-028-ITAJAI,...,Lactalis,442572,: 47996658453,Notebook,BRJ342B1N7,Tela do notebook danificada,NaN,INC_HD2_Field Services - BR,7700105,NaN
9,INC000001030864,FABIO AUGUSTO SILVA MORAIS LARA,BRA-078-PARA DE MINAS,Field Services - BR,BR_71 - Problema no equipamento,Pendente,26 de jun de 2024 13:16:14,FABIO AUGUSTO SILVA MORAIS LARA,FABIO AUGUSTO SILVA MORAIS LARA,BRA-078-PARA DE MINAS,...,Itambé,86041,: 37 991104065,Notebook,S/N BRJ401M9,Outros,: Ao colocar o computador na mochila o ziper d...,INC_HD2_Field Services - BR,3013400,NaN


In [10]:
dia = datetime.today().strftime('%d.%m')
df_pronto.to_excel(f'Chamados_BR71_{dia}.xlsx', index=False)